<h3>Primero tenemos que unificar, transformando si es necesario, los tres archivos de dataset que nos dieron:</h3>
1-Ver si son iguales, para descartar alguno, o analizar las diferencias que pudiesen existir<br>
2-ver si las cantidades de filas son iguales en los tres archivos<br>
3-ver si hay columnas faltantes, repetidas, o de igual nombre pero con diferentes campos<br><br>
empezamos por ver si son diferentes los dataset movies_dataset.xlsx, movies_dataset(1).xlsx,y movies_dataset.csv:

In [25]:
import pandas as pd
import warnings
import numpy as np
import csv


warnings.filterwarnings('ignore')#evitar comentarios de warnings, para verlo mas claramente

df_xlsx1 = pd.read_excel('Datasets/movies_dataset.xlsx')
df_xlsx2 = pd.read_excel('Datasets/movies_dataset_1.xlsx')
df_csv = pd.read_csv('Datasets/movies_dataset.csv', low_memory=True)

# Comparar xlsx1 con xlsx2
if df_xlsx1.equals(df_xlsx2):
    print("El dataframe de xlsx1 es exactamente igual al dataframe de xlsx2.")
else:
    print("<br>El dataframe de xlsx1 es diferente al dataframe de xlsx2.<br>")
    
# Comparar xlsx1 con csv
if df_xlsx1.equals(df_csv):
    print("El dataframe de xlsx1 es exactamente igual al dataframe de CSV")
else:
    print("El dataframe de xlsx1 es diferente al dataframe de CSV")

El dataframe de xlsx1 es exactamente igual al dataframe de xlsx2.
El dataframe de xlsx1 es diferente al dataframe de CSV


1-Descarto el archivo duplicado movies_dataset(1).xlsx ya que es igual al original<br><br>
2-Investigo esas diferencias entre los dos archivos restantes .XLSX y .CSV:

In [26]:
#VEO SUS DIMENSIONES SI COINCIDEN:

print(df_xlsx1.shape)
print(df_csv.shape)

#en filas y columnas no hay diferencias


(45466, 24)
(45466, 24)


In [27]:
df_xlsx1.head(1)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30 00:00:00,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,0.0,7.7,5415.0


In [28]:
df_csv.head(1)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0


Veo sus columnas y tipos de datos que contienen:<br>
-primero el dataframe del archivo CSV:

   Función para eliminar todas las comas consecutivas al final de una línea si hay 3 o más.<br>



    porque cuando lee EXCEL Files, las celdas del excel vacias en columnas ultimas que no tienen dato, pero
    si en esa columna de las miles de filas hay aunque sea un dato en esa columna en una fila, al leer el archivo 
    agrega esa columna al CSV como par de comas, representando la columna y campo vaciio para las miles de filas
    restantes solo por esa fila que si tiene un dato, las toma como dato.
    ENTONCES: hacemos un codigo que si encuentra comas al final de cada fila las elimine
    las filas deben terminar con un dato sin coma final, en el caso hipotetico de tener un caso que termine con ,,
    seria campo vacio en el CSV, este codigo daria error ya que tmb lo que hago en este codigo es unir saltos de linea
    incorrectos, que el excel interpreto como campos vacios los restantes de la linea, en vez de quitar automaticamente
    el salto de linea, y colocar en la linea actual la linea siguiente, al hacer la union, y al final hacer la conversion 
    de CSV a DataFrame deberia darme error si elimine un campo que terminaba en ,, , ya que en ese caso la extension de
    columnas no seria la misma para la totalidad de las filas, pero medio bien, asi que ningun problema


In [29]:
df_xlsx1 = pd.read_excel('Datasets/movies_dataset.xlsx',dtype=str,engine='openpyxl')
df_xlsx1.to_csv('Datasets/movies_dataset_antes_xlsx.csv',index=False)

df_xlsx1 = df_xlsx1.astype(str)
df_csv = df_csv.astype(str)
#----------------------------------------------------------------

with open('Datasets/movies_dataset.csv', 'r', encoding='utf-8') as file:
    lines = file.readlines()

cleaned_lines = []
i = 0

def remove_trailing_commas(line):
  
    while line.endswith(","):
        line = line[:-1]
    return line


while i < len(lines):
    line = lines[i].strip()
    # Verifica si la siguiente línea no comienza con 'True' o 'False' y no es la última línea
    while (i + 1) < len(lines) and not (lines[i + 1][:5] == "False" or lines[i + 1][:4] == "True"):
        i += 1  # Avanza a la siguiente línea
        line = remove_trailing_commas(line)
        line += lines[i].strip()  # Concatena la línea actual con la siguiente línea
    
    # Llama a la función para remover comas adicionales al final de la línea
    line = remove_trailing_commas(line)
    
    cleaned_lines.append(line + "\n")  # Añade la línea compuesta a la lista
    i += 1  # Avanza a la próxima línea

cleaned_content = "".join(cleaned_lines)

with open('Datasets/movies_dataset_cleaned.csv', 'w', encoding='utf-8') as file:
    file.write(cleaned_content)

#------------ahora con el XSXL--------------------------------------------------


with open('Datasets/movies_dataset_antes_xlsx.csv', 'r', encoding='utf-8') as file:
    lines = file.readlines()

cleaned_lines = []
i = 0

def remove_trailing_commas(line):
    """Función para eliminar todas las comas consecutivas al final de una línea si hay 3 o más."""
    while line.endswith(","):
        line = line[:-1]
    return line


while i < len(lines):
    line = lines[i].strip()   
    # Verifica si la siguiente línea no comienza con 'False' o 'True'
    while (i + 1) < len(lines) and not (lines[i + 1][:5] == "False" or lines[i + 1][:4] == "True"):
        i += 1  # Avanza a la siguiente línea
        line = remove_trailing_commas(line)
        line += lines[i].strip()  # Concatena la línea actual con la siguiente línea

    # Llama a la función para remover comas adicionales al final de la línea
    line = remove_trailing_commas(line)

    cleaned_lines.append(line + "\n")  # Añade la línea compuesta a la lista
    i += 1  # Avanza a la próxima línea

cleaned_content = "".join(cleaned_lines)

with open('Datasets/movies_dataset_antes_xlsx_cleaned.csv', 'w', encoding='utf-8') as file:
    file.write(cleaned_content)


#VEO SUS DIMENSIONES SI COINCIDEN:

print(df_xlsx1.shape)
print(df_csv.shape)

#como vemos tanto el csv, como el xlsx tenian estos errores 
#de salto de linea incorrectos, y se corrigio ahora claramente vemos
#3 menos lineas, pero ahora ,todas correctas, sin saltos de linea incorrectos"""


(45466, 24)
(45466, 24)


cambiar valores erroneos, columnas mal situadas en el orden de los dos dataframe tanto del archivo movies_dataset_cleaned.csv como del archivo movies_dataset_antes_xlsx.csv , todo esto luego de la lectura de los dataframe:

In [30]:

df_csv['revenue'] = df_csv['revenue'].str.split('.').str[0]
df_csv['runtime'] = df_csv['runtime'].str.split('.').str[0]
df_csv['release_date'] = df_csv['release_date'].str.split(' ').str[0]
df_csv['vote_count'] = df_csv['vote_count'].str.split('.0').str[0]
#df_csv['tagline'] = df_csv['tagline'].replace(r'^\s*$', 'Sin Dato', regex=True)
#df_csv['popularity'] = df_csv['popularity'].replace('Beware Of Frost Bites', 'Sin Dato')
#df_csv['video'] = df_csv['video'].apply(lambda x: x if (x == 'False' or x == 'True') else 'Sin Dato')
df_csv['original_title'] = df_csv['original_title'].str.lstrip('0')
df_csv['original_title'] = df_csv['original_title'].str.lstrip('0')
#df_csv['id'] = df_csv['id'].replace(r'^\s*$', 'Sin Dato', regex=True)


# Crear una nueva columna 'titles1'
df_csv['title1'] = df_xlsx1['title']
# Eliminar la columna 'titles' original
df_csv = df_csv.drop('title', axis=1)
# Renombrar la columna 'titles1' a 'titles'
df_csv.insert(20, 'title', df_csv['title1'])
# Eliminar la columna 'titles' original
df_csv = df_csv.drop('title1', axis=1)




df_csv['original_title'] = df_csv['original_title'].apply(lambda x: str(x)[1:] if str(x).startswith("'") else x)

df_csv['title'] = df_csv['title'].apply(lambda x: str(x)[1:] if str(x).startswith("'") else str(x))


#-----------------------------------------------------------------------------------------------------------------


df_xlsx1['popularity'] = df_xlsx1['popularity'].replace('Beware Of Frost Bites', '1234.1234')
df_xlsx1['popularity'] = df_xlsx1['popularity'].astype(float).astype(str)
df_xlsx1['popularity'] = df_xlsx1['popularity'].replace('1234.1234', 'Beware Of Frost Bites')


                                                        
df_xlsx1['vote_average'] = df_xlsx1['vote_average'].astype(float).astype(str)

#df_xlsx1['adult'] = df_xlsx1['adult'].apply(lambda x: x if x == 'False' else(x if x == 'True' else 'Sin Dato'))
df_xlsx1['revenue'] = df_xlsx1['revenue'].str.split('.').str[0]
df_xlsx1['runtime'] = df_xlsx1['runtime'].str.split('.').str[0]
df_xlsx1['release_date'] = df_xlsx1['release_date'].str.split(' ').str[0]
#df_xlsx1['video'] = df_xlsx1['video'].apply(lambda x: x if (x == 'False' or x =='True') else 'Sin Dato')


df_xlsx1['vote_count'] = df_xlsx1['vote_count'].str.split('.0').str[0]

df_xlsx1['original_title'] = df_csv['original_title'].str.lstrip('0')
#df_xlsx1['id'] = df_xlsx1['id'].replace(r'^\s*$', 'Sin Dato', regex=True)

# Crear una nueva columna 'titles1'
df_xlsx1['original_title1'] = df_csv['original_title']
# Eliminar la columna 'titles' original
df_xlsx1 = df_xlsx1.drop('original_title', axis=1)
# Renombrar la columna 'titles1' a 'titles'
df_xlsx1.insert(8, 'original_title', df_xlsx1['original_title1'])
# Eliminar la columna 'titles' original
df_xlsx1 = df_xlsx1.drop('original_title1', axis=1)

df_xlsx1['overview'] = df_csv['overview']
        
# recorrer cada fila
for i in range(len(df_csv)):
    if df_csv.loc[i, 'tagline'].startswith("'"):
        df_xlsx1.loc[i, 'tagline'] = "'" + df_xlsx1.loc[i, 'overview']
    
for x in df_csv['tagline']:
            if x != 'Sin Dato':
                 df_xlsx1['tagline'] = df_csv['tagline']


#datos investigados outliers, e investigados por la web, de cuales esran los certeros entre los dos archivos
#df_xlsx1['video'] = df_csv['video']
df_xlsx1['release_date']=df_csv['release_date']


Ahora nuevamente trato de Encontrar las filas y columnas donde los dataframes de cadena difieren, a traves del metodo ".ne" luego de las previas trasnformaciones, por si quedaron datos que no concuerdan entre dataframes:

In [31]:

differences_str = df_csv.ne(df_xlsx1)#ne 'not equal' devuelve un fdataframe misma dimension que los dos a comparar,
                                     #da True si hay diferencia entre data frames y False si los valores
                                     #LOS CAMPOS DE TODO EL DATAFRAME DEBEN SER DEL MISMO TIPO, EN ESTE CASO STRING
                                     #PARA LOS DOS DATAFRAMES A COMPARAR SINO DA DIFERENTE AUNQUE SEA EL MISMO VALOR
                                     # Para encontrar las ubicaciones exactas donde los dataframes de cadena difieren
#------------------------------------------------------------------------------------------
different_locations_str = np.where(df_csv != df_xlsx1)
""" np.where(df_csv != df_xlsx1) 
devuelve un arreglo de índices que corresponden a las ubicaciones de todos los 
elementos en df_csv y df_xlsx1 que son diferentes."""

# Para obtener los valores de esas ubicaciones en cada dataframe
values_df1_str = df_csv.values[different_locations_str]
values_df2_str = df_xlsx1.values[different_locations_str]

#-----------------------------------------------------------------------------------------
# Para ver las diferencias por columnas
column_differences_str = differences_str.sum()#TODAS LAS DIFERENCIAS
"""
La función sum() aplicada a un DataFrame retorna una serie con la suma 
de los valores para cada columna. Entonces, si lo aplicamos a differences_str, 
que es un DataFrame que contiene valores booleanos 
(True si hay una diferencia en la ubicación correspondiente, False de lo contrario), 
la suma de una columna dará el número de Trues en esa columna, que corresponde al número 
de diferencias en esa columna entre los dos DataFrames."""
#--------------------------------------------------------------------------------------------
# Para ver el total de diferencias
total_differences_str = differences_str.sum().sum()

print(f"Las ubicaciones donde difieren los dos dataframes son: {different_locations_str}")
print(f"Los valores en esas ubicaciones en df1 son: {values_df1_str}")
print(f"Los valores en esas ubicaciones en df2 son: {values_df2_str}")
print(f"Las diferencias por columnas son: \n{column_differences_str}")
print(f"El total de diferencias es: {total_differences_str}")

#pd.set_option('display.max_colwidth', 100)

if different_locations_str[0].size != 0:
    
    first_difference_row = different_locations_str[0][0]

    # Muestra esa fila de cada DataFrame
    print("La primera fila con diferencias en df1 es:")
    df1_row = pd.DataFrame(df_csv.iloc[first_difference_row]).transpose()
    display(df1_row)

    print("\nLa primera fila con diferencias en df2 es:")
    df2_row = pd.DataFrame(df_xlsx1.iloc[first_difference_row]).transpose()
    display(df2_row)# Encuentra la primera columna con una diferencia
    first_difference_column = differences_str.any().idxmax()

    # Crea un nuevo DataFrame con las columnas en cuestión de ambos dataframes originales
    df_combined = pd.merge(df_csv[[first_difference_column]], df_xlsx1[[first_difference_column]], left_index=True, right_index=True, suffixes=('_csv', '_xlsx'))

    # Filtra el DataFrame combinado para solo las filas donde las dos columnas no son iguales
    df_differences = df_combined[df_combined[first_difference_column + '_csv'] != df_combined[first_difference_column + '_xlsx']]

    # Muestra el DataFrame
    display(df_differences)
else:
    print("\nNo hay diferencias entre los dataframes.")
    




Las ubicaciones donde difieren los dos dataframes son: (array([19730, 19730, 19730, 19730, 29503, 29503, 29503, 29503, 35587,
       35587, 35587], dtype=int64), array([ 0,  5,  7, 13,  0,  5,  7, 13,  0,  5,  7], dtype=int64))
Los valores en esas ubicaciones en df1 son: [' - Written by ' '1997-08-20' '104.0' '6.0'
 ' Rune Balot goes to a casino connected to the October corporation to try to wrap up her case once and for all.'
 '2012-09-29' '68.0' '7.0'
 ' Avalanche Sharks tells the story of a bikini contest that turns into a horrifying affair when it is hit by a shark avalanche.'
 '2014-01-01' '82.0']
Los valores en esas ubicaciones en df2 son: ['- Written by Ørnås' '1997-08-20 00:00:00' '104' '6'
 'Rune Balot goes to a casino connected to the October corporation to try to wrap up her case once and for all.'
 '2012-09-29 00:00:00' '68' '7'
 'Avalanche Sharks tells the story of a bikini contest that turns into a horrifying affair when it is hit by a shark avalanche.'
 '2014-01-01 00:00

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
19730,- Written by,0.065736,/ff9qCepilowshEtG2GYWwzt2bs4.jpg,"[{'name': 'Carousel Productions', 'id': 11176}...","[{'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso...",1997-08-20,0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,...,1,nan,nan,nan,nan,nan,nan,nan,nan,nan



La primera fila con diferencias en df2 es:


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
19730,- Written by Ørnås,0.065736,/ff9qCepilowshEtG2GYWwzt2bs4.jpg,"[{'name': 'Carousel Productions', 'id': 11176}...","[{'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso...",1997-08-20 00:00:00,0,104,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,...,1,nan,nan,nan,nan,nan,nan,nan,nan,nan


,adult_csv,adult_xlsx
19730,- Written by,- Written by Ørnås
29503,Rune Balot goes to a casino connected to the ...,Rune Balot goes to a casino connected to the O...
35587,Avalanche Sharks tells the story of a bikini ...,Avalanche Sharks tells the story of a bikini c...


<h4>Finalmente ya unificamos los datasets, y creamos un único dataset final.csv para seguir con las transformaciones pedidas</h4>

In [32]:
df_csv.to_csv('final.csv',index=False)